In [ ]:
from linkedin_scraper import get_links, links_to_df
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import sqlite3

### Getting our data

In [ ]:
URL = 'https://br.linkedin.com/jobs'
job_keyword = 'Data Scientist'
driver = webdriver.Chrome(executable_path='/Users/pedrodib/Python/Linkedin Webscraping/chromedriver')

# Looping through the links list to get some info about the jobs and saving them into a dictionary
links = get_links(URL, job_keyword, driver)
jobs_dataframe = links_to_df(job_keyword, links)

# Making sure we take only the values whose title are different from null
jobs_dataframe = jobs_dataframe[jobs_dataframe['title'].notna()]

# Checking if there are duplicates
jobs_dataframe = jobs_dataframe.drop_duplicates(subset=['description'])

### Storing the data

In [ ]:
# Setting the database
# TODO: migrate the database from SQLite to a PostgreSQL table
DATABASE_LOCATION = 'sqlite:///new_jobs_database.sqlite'

# Loading the data
engine = sqlalchemy.create_engine(DATABASE_LOCATION)
conn = sqlite3.connect('new_jobs_database.sqlite')
cursor = conn.cursor()

# Let's create a table with some basic structure
sql_query = '''
CREATE TABLE IF NOT EXISTS jobs_database(
    keyword VARCHAR(30) NOT NULL,
    title VARCHAR(60),
    company_name VARCHAR(30),
    location VARCHAR(60),
    description VARCHAR(8000) NOT NULL,
    link VARCHAR(200) NOT NULL,
    CONSTRAINT primary_key_constraint PRIMARY KEY (description)
    )
    '''

# Executing the query we have hust created
cursor.execute(sql_query)
print('Opened database successfully.')

# Appending the new values
jobs_dataframe.to_sql('jobs_database', engine, index=False, if_exists='append')
print('Values inserted into database.')

# Closing the database
cursor.close()
print('Closed database successfully.')